In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

train_input = pd.read_csv('/Users/aran/Desktop/Semester 1/Introduction to machine learning/assignments/SVD assignment/Data/trainInput.csv', header = None).values
train_output = pd.read_csv('/Users/aran/Desktop/Semester 1/Introduction to machine learning/assignments/SVD assignment/Data/trainOutput.csv', header = None).values.flatten()

test_input = pd.read_csv('/Users/aran/Desktop/Semester 1/Introduction to machine learning/assignments/SVD assignment/Data/testInput.csv', header = None).values
test_output = pd.read_csv('/Users/aran/Desktop/Semester 1/Introduction to machine learning/assignments/SVD assignment/Data/testOutput.csv', header = None).values.flatten()

# Transpose train_input and test_input so each row is an image
train_input = train_input.T  # Now shape is (1707, 256)
test_input = test_input.T  # Now shape is (2007, 256)

# Step 2: Separate images by digit (each row represents an image)
digit_matrices = {digit: train_input[train_output == digit, :] for digit in range(10)}

# Confirm that each digit's matrix has the shape (n_samples_for_digit, 256)
for digit, matrix in digit_matrices.items():
    print(f"Digit {digit}: matrix shape = {matrix.shape}")  # Expected (n_samples_for_digit, 256)

# Step 3: Perform SVD for each digit matrix and keep the first 20 right singular vectors
k = 20
singular_vectors = {}

for digit, matrix in digit_matrices.items():
    # Perform SVD, where U * Sigma * Vt = matrix
    _, _, Vt = np.linalg.svd(matrix, full_matrices=False)
    singular_vectors[digit] = Vt[:k, :]  # Shape (20, 256)

# Verify the shapes of the singular vectors for each digit
for digit, basis in singular_vectors.items():
    print(f"Digit {digit} basis shape: {basis.shape}")  # Expected (20, 256)

# Step 4: Classify each test image by projecting it onto each digit's singular vectors
predictions = []

for test_image in test_input:
    residuals = []
    for digit in range(10):
        basis = singular_vectors[digit]  # Shape: (20, 256)

        # Solve for coefficients in the least squares sense
        coefficients = np.linalg.lstsq(basis.T, test_image, rcond=None)[0]
        
        # Approximate the image using the basis and coefficients
        approximation = basis.T @ coefficients
        
        # Calculate the residual (Euclidean distance) between the test image and its approximation
        residual = np.linalg.norm(test_image - approximation)
        residuals.append(residual)
    
    # Classify as the digit with the smallest residual
    predicted_digit = np.argmin(residuals)
    predictions.append(predicted_digit)

# Step 5: Evaluate classification accuracy
accuracy = accuracy_score(test_output, predictions)
print(f"Overall Classification Accuracy: {accuracy * 100:.2f}%")

# Step 6: Generate and display confusion matrix
conf_matrix = confusion_matrix(test_output, predictions)
print("Confusion Matrix:")
print(conf_matrix)


Digit 0: matrix shape = (319, 256)
Digit 1: matrix shape = (252, 256)
Digit 2: matrix shape = (202, 256)
Digit 3: matrix shape = (131, 256)
Digit 4: matrix shape = (122, 256)
Digit 5: matrix shape = (88, 256)
Digit 6: matrix shape = (151, 256)
Digit 7: matrix shape = (166, 256)
Digit 8: matrix shape = (144, 256)
Digit 9: matrix shape = (132, 256)
Digit 0 basis shape: (20, 256)
Digit 1 basis shape: (20, 256)
Digit 2 basis shape: (20, 256)
Digit 3 basis shape: (20, 256)
Digit 4 basis shape: (20, 256)
Digit 5 basis shape: (20, 256)
Digit 6 basis shape: (20, 256)
Digit 7 basis shape: (20, 256)
Digit 8 basis shape: (20, 256)
Digit 9 basis shape: (20, 256)
Overall Classification Accuracy: 93.97%
Confusion Matrix:
[[355   0   2   0   1   0   0   0   0   1]
 [  0 259   0   0   3   0   2   0   0   0]
 [  8   1 179   2   3   0   0   1   4   0]
 [  1   0   4 148   1   8   0   1   2   1]
 [  1   1   0   0 187   1   1   3   0   6]
 [  8   1   2   4   0 140   0   0   2   3]
 [  2   0   0   0   2   2